In [2]:
library(pROC)
library(readxl)
library(xlsx)
library(gridExtra)
library(ggplot2)
library(parallel)

In [3]:
setwd("..")

In [4]:
sampleAnnotFile = "annotation/annotation-78.csv"
pcrValueFile = "data/PCR/PCR-measurement_78.xlsx"
outDir = "Figures-and-Tables/"
tableS6outfile = "Table-S6-model-crossvalidation-results.csv"

In [5]:
# sample annotation
annot = read.csv(file = sampleAnnotFile ,header = T,sep = "\t",stringsAsFactors = F)
annot$gender = ifelse(annot$gender=="f","female","male")
head(annot,3)

,ID,gender,age,group
,<chr>,<chr>,<int>,<chr>
1,SXR0002,female,80,ccRCC
2,SXR0004,male,50,ccRCC
3,SXR0006,male,68,ccRCC


### read in PCR data
and normalize molec. counts to house keeping gene

In [6]:
pcr = as.data.frame(read_excel(path = pcrValueFile, sheet = 1))
pcr$SNORD99_ACTB_mol = pcr$SNORD99_molec_count/pcr$ACTB_molec_count
pcr = pcr[,c("ID","SNORD99_ACTB_mol")]
head(pcr,2)

,ID,SNORD99_ACTB_mol
,<chr>,<dbl>
1,SXR0002,0.25324223
2,SXR0004,0.09088467


In [7]:
tmp = as.data.frame(read_excel(path = pcrValueFile, sheet = 2))
tmp$SNORD22_RNY3_mol = tmp$SNORD22_molec_count/tmp$RNY3_molec_count
tmp$SNORD26_RNY3_mol = tmp$SNORD26_molec_count/tmp$RNY3_molec_count
tmp = tmp[,c("ID","SNORD22_RNY3_mol","SNORD26_RNY3_mol")]
pcr = merge(pcr,tmp,by="ID")
head(pcr,2)

,ID,SNORD99_ACTB_mol,SNORD22_RNY3_mol,SNORD26_RNY3_mol
,<chr>,<dbl>,<dbl>,<dbl>
1,SXR0002,0.25324223,Inf,NaN
2,SXR0004,0.09088467,0.06216743,0.01914862


In [8]:
tmp = as.data.frame(read_excel(path = pcrValueFile, sheet = 3))
tmp$SNORA50C_ACTB_mol = tmp$SNORA50C_molec_count / tmp$ACTB_molec_count
tmp = tmp[,c("ID","SNORA50C_ACTB_mol")]
pcr = merge(pcr,tmp,by="ID")
head(pcr,3)

,ID,SNORD99_ACTB_mol,SNORD22_RNY3_mol,SNORD26_RNY3_mol,SNORA50C_ACTB_mol
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,SXR0002,0.25324223,Inf,NaN,0.021118913
2,SXR0004,0.09088467,0.06216743,0.01914862,0.004867978
3,SXR0006,0.02808557,0.03330643,0.01758783,0.012005747


In [9]:
pcr = merge(x = pcr,y=annot, by = "ID", all.y=F)
pcr$SNORD99_ACTB_mol[which(is.infinite(pcr$SNORD99_ACTB_mol) | is.nan(pcr$SNORD99_ACTB_mol))] = NA
pcr$SNORD22_RNY3_mol[which(is.infinite(pcr$SNORD22_RNY3_mol) | is.nan(pcr$SNORD22_RNY3_mol))] = NA
pcr$SNORD26_RNY3_mol[which(is.infinite(pcr$SNORD26_RNY3_mol) | is.nan(pcr$SNORD26_RNY3_mol))] = NA
pcr$SNORA50C_ACTB_mol[which(is.infinite(pcr$SNORA50C_ACTB_mol) | is.nan(pcr$SNORA50C_ACTB_mol))] = NA
head(pcr,3)

,ID,SNORD99_ACTB_mol,SNORD22_RNY3_mol,SNORD26_RNY3_mol,SNORA50C_ACTB_mol,gender,age,group
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<chr>
1,SXR0002,0.25324223,NA,NA,0.021118913,female,80,ccRCC
2,SXR0004,0.09088467,0.06216743,0.01914862,0.004867978,male,50,ccRCC
3,SXR0006,0.02808557,0.03330643,0.01758783,0.012005747,male,68,ccRCC


In [10]:
pcr$group_binary = factor(x = pcr$group, levels = c("urolithiasis","ccRCC"))
rownames(pcr) = pcr$ID

In [11]:
allGeneVars = c("SNORD99_ACTB_mol","SNORD22_RNY3_mol","SNORD26_RNY3_mol","SNORA50C_ACTB_mol")
names(allGeneVars) = c("SNORD99","SNORD22","SNORD26","SNORA50C")

### find valuable random partition of data

In [12]:
# create a random partition of the data
createPartition = function(k=5, dat) {
    partLen = nrow(dat)/k # 5 partitions of ca 16 samples each
    idx = 1:nrow(dat)
    idxShuffPart = split(sample(idx,length(idx)),ceiling(seq_along(idx) / partLen))
    return(unname(idxShuffPart))
}

In [13]:
# for a partition of data, print the number of each group (ccRCC, uro)
printGroupPartition = function(dat, partition, var="group") {
    for (part in partition) {
        t = table(dat[part,var])
        print(t)
    }
}

In [14]:
# check that in each partition there are at least minA and minB samples
#   in the groups of variable var
#   groupA and groupB are the names of the groups in var
checkPartition = function(dat, partition, var="group",groupA,groupB,minA, minB) {
    isOK = T
    for (part in partition) {
        lengthA = length(which(dat[part,var] == groupA))
        lengthB = length(which(dat[part,var] == groupB))
        goodOne = lengthA >= minA & lengthB >= minB
        isOK = isOK & goodOne
    }
    return(isOK)
}

In [15]:
# call functions above: create random partitions until a right one is found
findPartition = function(maxIter,k,dat=dat,var = "group",groupA,groupB,minA,minB) {
    end = F; i = 0;
    partition=NULL
    while(!end) {
        partition = createPartition(k = k, dat = dat)
        check = checkPartition(dat = pcr, partition = partition, var=var,
                   groupA = groupA,groupB = groupB,minA = minA,minB = minB)
        i = i + 1
        end = i > maxIter | check
    }
    return(partition)
}

In [16]:
# create and evaluate regression model from/with training and test data

glmFlexible = function(trainDat, testDat, variables = c("SNORD99_ACTB_mol","SNORD22_RNY3_mol"),
                             confounders = c("age","gender")) {
    varValuesTrain = sapply(variables, function(var) 
        log(trainDat[,var] + min(setdiff(trainDat[,var],0), na.rm = T)/2))
    varValuesTest = sapply(variables, function(var) 
        log(testDat[,var] + min(setdiff(testDat[,var],0), na.rm = T)/2))
    colnames(varValuesTrain) = paste0("var_",1:length(variables))
    trainDat = cbind(trainDat, varValuesTrain)
    colnames(varValuesTest) = paste0("var_",1:length(variables))
    testDat = cbind(testDat, varValuesTest)
    
    # don't run if there are not both ccRCC and controls present in either test or training data:
    if(length(table(testDat[,"group_binary"]))==1 |
      length(table(trainDat[,"group_binary"]))==1 ) {
        return(NULL)
    }
    formula = as.formula(paste0("group_binary ~ ",paste0(colnames(varValuesTrain),collapse=" + ")," + ",
           paste0(confounders,collapse=" + ")))
    # remove rows where any variable is NA
    wh = which(!apply(trainDat[,c(colnames(varValuesTrain),confounders), drop=F],1, function(l) any(is.na(l))))
    trainDat = trainDat[wh,,drop=F]
    wh = which(!apply(testDat[,c(colnames(varValuesTest),confounders), drop=F],1, function(l) any(is.na(l))))
    testDat = testDat[wh,,drop=F]
    
    glmModel = glm(data = trainDat,formula = formula, family = "binomial")   
    # p-value of whole model, i.e. better than a null model?
    # from https://stats.stackexchange.com/questions/129958/glm-in-r-which-pvalue-represents-the-goodness-of-fit-of-entire-model
                      
    nullModel = glm(formula = "group_binary ~ 1",data = trainDat, family = "binomial")   
    p_model = with(anova(nullModel,glmModel),pchisq(Deviance,Df,lower.tail=FALSE)[2]) 

    testProb = predict(glmModel, newdata = testDat, type = "response") 
    label = paste0(paste0(names(variables),collapse=" + ")," + ",paste0(confounders,collapse=" + "))
    testRoc = roc(group_binary ~ testProb, data = testDat , plot = F, print.auc = TRUE, 
                      print.auc.cex=1.2, main = label, cex.main=0.9, quiet=T)

    # best cutoff:
    wh = which.max(testRoc$sensitivities + testRoc$specificities)
    cutoff = testRoc$thresholds[wh]
    confusion = table(ifelse(testProb>cutoff, 1, 0), ifelse(testDat$group_binary=="ccRCC",1,0))
    confusion = confusion[c("1","0"),c("1","0")]
                      
    D = confusion[2,2]; C = confusion[2,1]; B = confusion[1,2]; A = confusion[1,1]
    accuracy = (A+D)/(A+B+C+D)
    sens = A / (A + C)
    spec = D / (D + B)
                      
                      
    stats = data.frame(p_model = p_model, sens = sens, spec=spec, acc=accuracy, AUC=testRoc$auc, 
            stringsAsFactors = F)
    rownames(stats) = label

    return(stats)
}

In [17]:
# for a given data partition, calculate all models/metrics of all possible partitions of the training/test data
#  that is, cycle through the 5 partition bins 
callGlmPermutations = function(dat, partition, variables, confounders) {
    allStats = NULL
    for (i in 1:length(partition)) {
        testIdx = partition[[i]]
        testDat = dat[testIdx,]
        trainIdx = setdiff(unlist(partition),partition[[i]])
        trainDat = dat[trainIdx,]
        stat = glmFlexible(trainDat = trainDat, testDat = testDat,variables = variables,
                      confounders = confounders)
        if(is.null(stat)) {next}
        pNames = grep("p_\\d+",colnames(stat),value = T)
        allStats = rbind(allStats, data.frame( row.names = rownames(stat), stringsAsFactors = F,
            Sensitivity=stat$sens, Specificity=stat$spec,
            Accuracy=stat$acc, AUC=stat$AUC, p_model=stat$p_model))
    }    
    return(allStats)
}

In [18]:
# parallelized calling repeatedly the function for calculation of all permutations of a given partition
callGlmPermutationsRepeatedlyParallel = function(
    nbCPUs = 40, nbRepeats = 1000, dat, variables, confounders, 
    k = 5, var = "group", groupA = "ccRCC", groupB = "urolithiasis",
    minA = 10,minB = 4) {
    
    maxIterPartition = 3000 # how often do we try until we give up finding a partition
    
    oneCall = function(maxIter,k,dat, var, groupA, groupB,minA,minB) {
        partition = findPartition(maxIter = maxIter,k = k,dat = dat, var = var, 
                          groupA = groupA, groupB = groupB,minA = minA,minB = minB)
        if(!is.null(partition)) {
            stats  = callGlmPermutations(dat = dat, partition = partition, 
                            variables = variables, confounders = confounders)
        } else {
            stats = NULL
        }
        return(stats)
    }
    
    stats = mclapply(X = 1:nbRepeats, mc.cores = nbCPUs,
             FUN = function(x) oneCall(maxIter = maxIterPartition,k = k,dat = dat, var = var, 
                          groupA = groupA, groupB = groupB,minA = minA,minB = minB))
    stats = do.call(rbind,stats) # do.call omits NULL automatically
    message(nrow(stats)," iterations managed")
    m = apply(stats,2,function(c) mean(c,na.rm=T))
    s = apply(stats,2,function(c) sd(c,na.rm=T))
    rbind(m,s)
    stats = data.frame(formula = rownames(stats)[1],t(as.vector(rbind(m,s))))

    colnames(stats) = c("formula", "sens_mean","sens_SD","spec_mean","spec_SD",
                        "ACC_mean","ACC_SD","ACU_mean","ACU_SD","p_model_mean","p_model_SD")
    stats
}

# 1 gene models

In [19]:
allStats = NULL
allStats = rbind(allStats,
                 callGlmPermutationsRepeatedlyParallel(# nbCPUs = 40, nbRepeats = 100,
                       dat = pcr, variables = allGeneVars[c("SNORD99")], confounders = c("age","gender"), 
                       k = 5, var = "group",groupA = "ccRCC", groupB = "urolithiasis",minA = 10,minB = 4))

5000 iterations managed



In [20]:
allStats = rbind(allStats,
                 callGlmPermutationsRepeatedlyParallel(#nbCPUs = 60, nbRepeats = 100,
                       dat = pcr, variables = allGeneVars[c("SNORD22")], confounders = c("age","gender"), 
                       k = 5, var = "group",groupA = "ccRCC", groupB = "urolithiasis",minA = 10,minB = 4))

5000 iterations managed



In [21]:
allStats = rbind(allStats,
                 callGlmPermutationsRepeatedlyParallel(#nbCPUs = 60, nbRepeats = 100,
                       dat = pcr, variables = allGeneVars[c("SNORD26")], confounders = c("age","gender"), 
                       k = 5, var = "group",groupA = "ccRCC", groupB = "urolithiasis",minA = 10,minB = 4))

5000 iterations managed



In [22]:
allStats = rbind(allStats,
                 callGlmPermutationsRepeatedlyParallel(#nbCPUs = 60, nbRepeats = 100,
                       dat = pcr, variables = allGeneVars[c("SNORA50C")], confounders = c("age","gender"), 
                       k = 5, var = "group",groupA = "ccRCC", groupB = "urolithiasis",minA = 10,minB = 4))

5000 iterations managed



# 2 gene models

In [23]:
allStats = rbind(allStats,
                 callGlmPermutationsRepeatedlyParallel(#nbCPUs = 60, nbRepeats = 100,
                       dat = pcr, variables = allGeneVars[c("SNORD99","SNORD22")], confounders = c("age","gender"), 
                       k = 5, var = "group",groupA = "ccRCC", groupB = "urolithiasis",minA = 10,minB = 4))

5000 iterations managed



In [24]:
allStats = rbind(allStats,
                 callGlmPermutationsRepeatedlyParallel(#nbCPUs = 60, nbRepeats = 100,
                       dat = pcr, variables = allGeneVars[c("SNORD99","SNORD26")], confounders = c("age","gender"), 
                       k = 5, var = "group",groupA = "ccRCC", groupB = "urolithiasis",minA = 10,minB = 4))

5000 iterations managed



In [25]:
allStats = rbind(allStats,
                 callGlmPermutationsRepeatedlyParallel(#nbCPUs = 60, nbRepeats = 100,
                       dat = pcr, variables = allGeneVars[c("SNORD99","SNORA50C")], confounders = c("age","gender"), 
                       k = 5, var = "group",groupA = "ccRCC", groupB = "urolithiasis",minA = 10,minB = 4))

5000 iterations managed



In [26]:
allStats = rbind(allStats,
                 callGlmPermutationsRepeatedlyParallel(#nbCPUs = 60, nbRepeats = 100,
                       dat = pcr, variables = allGeneVars[c("SNORD22","SNORD26")], confounders = c("age","gender"), 
                       k = 5, var = "group",groupA = "ccRCC", groupB = "urolithiasis",minA = 10,minB = 4))

5000 iterations managed



In [27]:
allStats = rbind(allStats,
                 callGlmPermutationsRepeatedlyParallel(#nbCPUs = 60, nbRepeats = 100,
                       dat = pcr, variables = allGeneVars[c("SNORD22","SNORA50C")], confounders = c("age","gender"), 
                       k = 5, var = "group",groupA = "ccRCC", groupB = "urolithiasis",minA = 10,minB = 4))

5000 iterations managed



In [28]:
allStats = rbind(allStats,
                 callGlmPermutationsRepeatedlyParallel(#nbCPUs = 60, nbRepeats = 100,
                       dat = pcr, variables = allGeneVars[c("SNORD26","SNORA50C")], confounders = c("age","gender"), 
                       k = 5, var = "group",groupA = "ccRCC", groupB = "urolithiasis",minA = 10,minB = 4))

5000 iterations managed



# 3 gene models

In [29]:
allStats = rbind(allStats,
                 callGlmPermutationsRepeatedlyParallel(#nbCPUs = 60, nbRepeats = 100,
                       dat = pcr, variables = allGeneVars[c("SNORD99","SNORD22","SNORD26")], confounders = c("age","gender"), 
                       k = 5, var = "group",groupA = "ccRCC", groupB = "urolithiasis",minA = 10,minB = 4))

5000 iterations managed



In [30]:
allStats = rbind(allStats,
                 callGlmPermutationsRepeatedlyParallel(#nbCPUs = 60, nbRepeats = 100,
                       dat = pcr, variables = allGeneVars[c("SNORD99","SNORD22","SNORA50C")], confounders = c("age","gender"), 
                       k = 5, var = "group",groupA = "ccRCC", groupB = "urolithiasis",minA = 10,minB = 4))

5000 iterations managed



In [31]:
allStats = rbind(allStats,
                 callGlmPermutationsRepeatedlyParallel(#nbCPUs = 60, nbRepeats = 100,
                       dat = pcr, variables = allGeneVars[c("SNORD99","SNORD26","SNORA50C")], confounders = c("age","gender"), 
                       k = 5, var = "group",groupA = "ccRCC", groupB = "urolithiasis",minA = 10,minB = 4))

5000 iterations managed



In [32]:
allStats = rbind(allStats,
                 callGlmPermutationsRepeatedlyParallel(#nbCPUs = 60, nbRepeats = 100,
                       dat = pcr, variables = allGeneVars[c("SNORD22","SNORD26","SNORA50C")], confounders = c("age","gender"), 
                       k = 5, var = "group",groupA = "ccRCC", groupB = "urolithiasis",minA = 10,minB = 4))

5000 iterations managed



# 4 gene model

In [33]:
allStats = rbind(allStats,
                 callGlmPermutationsRepeatedlyParallel(#nbCPUs = 60, nbRepeats = 100,
                       dat = pcr, variables = allGeneVars[c("SNORD99","SNORD22","SNORD26","SNORA50C")], confounders = c("age","gender"), 
                       k = 5, var = "group",groupA = "ccRCC", groupB = "urolithiasis",minA = 10,minB = 4))

5000 iterations managed



In [34]:
allStats

formula,sens_mean,sens_SD,spec_mean,spec_SD,ACC_mean,ACC_SD,ACU_mean,ACU_SD,p_model_mean,p_model_SD
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
SNORD99 + age + gender,0.6464391,0.2695536,0.6205267,0.2689612,0.6381150,0.2006236,0.6493684,0.1198119,0.08192688,0.08190804
SNORD22 + age + gender,0.6584364,0.2585946,0.6666800,0.2712829,0.6605614,0.1891324,0.6632008,0.1238483,0.11934673,0.10859599
SNORD26 + age + gender,0.6596755,0.2825902,0.6188133,0.2495259,0.6449299,0.2103540,0.6499003,0.1200608,0.14484891,0.11960608
SNORA50C + age + gender,0.7321909,0.2306254,0.6994533,0.2286998,0.7216333,0.1689294,0.7023139,0.1263403,0.01911011,0.02460023
SNORD99 + SNORD22 + age + gender,0.6484494,0.2760701,0.6260233,0.2745516,0.6399347,0.2019171,0.6484732,0.1213352,0.12678067,0.11559698
SNORD99 + SNORD26 + age + gender,0.6545997,0.3086018,0.5750467,0.2355407,0.6264555,0.2285751,0.6427436,0.1216194,0.13886990,0.12039499
SNORD99 + SNORA50C + age + gender,0.7172079,0.2353600,0.6851333,0.2360492,0.7070158,0.1744832,0.6883508,0.1274113,0.03600384,0.04455280
SNORD22 + SNORD26 + age + gender,0.6593769,0.2838945,0.6372333,0.2569604,0.6512688,0.2042592,0.6548757,0.1241972,0.12402268,0.11613991
SNORD22 + SNORA50C + age + gender,0.7023180,0.2632412,0.6646767,0.2427793,0.6888127,0.1883857,0.6761093,0.1282387,0.05166399,0.06013000


In [35]:
outf = paste0(outDir,tableS6outfile)
outf
write.table(x = allStats, file = outf, quote = F, sep = "\t", row.names = F)

[1] "Figures-and-Tables/Table-S6-model-crossvalidation-results.csv"